In [ ]:
# Pyton Package Installation

## virtual environment setup
%pip install conda

## nlp libraries
%pip install --upgrade gensim
%pip install top2vec

## machine learning libraries
%pip install transformers
%pip install torch

## visualization libraries
%pip install matplotlib
%pip install seaborn
%pip install pyldavis
%pip install cartopy

## datahandling libraries
%pip install pandas

SyntaxError: invalid syntax (1209796400.py, line 18)

In [ ]:
# Verwendete Bibliotheken
%pip install bertopic
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# NLP-Pipeline
import nltk 
import sklearn
import gensim
# import spaCy as sp
import re

# ML
import transformers
import torch 

## Embeddings

import top2vec as t2v 

## Visualisierung
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud as wc
import pyLDAvis
import cartopy as ctp

# Sortieren
import venv
import numpy as np

TF-IDF

In [ ]:
%pip install spacy
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl
%pip uninstall transformers torch sentencepiece -y

Note: you may need to restart the kernel to use updated packages.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
Note: you may need to restart the kernel to use updated packages.
Found existing installation: transformers 4.57.5
Uninstalling transformers-4.57.5:
  Successfully uninstalled transformers-4.57.5
Found existing installation: torch 2.9.1
Uninstalling torch-2.9.1:
  Successfully uninstalled torch-2.9.1
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
Note: you may need to restart the kernel to use updated packages.


# Datenvorverarbeiten (engl. data pre-processing)

In [ ]:
# Landen des bereinigten Datensatzes
df = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv', usecols=["text"])
print(df.head(10))
print(df.shape)

                                                text
0  I used to love Comcast. Until all these consta...
1  I'm so over Comcast! The worst internet provid...
2  If I could give them a negative star or no sta...
3  I've had the worst experiences so far since in...
4  Check your contract when you sign up for Comca...
5  Thank God. I am changing to Dish. They gave me...
6  I Have been a long time customer and only have...
7  There is a malfunction on the DVR manager whic...
8  Charges overwhelming. Comcast service rep was ...
9  I have had cable, DISH, and U-verse, etc. in t...
(5627, 1)


In [29]:
import pandas as pd
import spacy

# Textbereinigung (engl. text cleaning)
##  Rauschentfernung (engl. noise reduction)
### Wortbereinigung (engl. word cleaning)

# Datensatz laden (Rohtext)
df = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv', 
                 usecols=["text"],nrows=5627)                                # Zeilenbegrenzung

# Linguistische Verarbeitung (engl. linguistic processing)
## Rechtschreibfehlerkorrektur (engl. spelling correction)

## Textbereinigung, Tokenisierung, Lemmatisierung

## spaCy Pipeline
nlp = spacy.load("en_core_web_sm")                                          # englisches Modell (small version)

## Textbereinigung, Tokenisierung, Grundformreduktion (Lemmatisierung)
df["cleaned"] = [[token.lemma_.lower()
                  for token in doc
                  # Filterblock      
                  if (not token.is_stop and                                 # Standard:     Stopwort-Filter
                      not token.is_punct and                                # Standard:     Satzzeichen-Filter
                      not token.like_num and                                # Standard:     Nummern-Filter (einfache Zahlen)
                      len(token.text) > 2 and                               # Individuell:  Wörter mit min. 2 Zeichen
                      not any(char in token.text for char in ':/-–—') and   # Individuell:  Filter für Datums-/Zeit-Token
                      token.is_ascii and                                    # Individuell:  Emoijs-Filter (True = AN / False = Nur Emoijs)
                      token.pos_ != "PRON" and                              # Individuell:  Pronomen-Filter
                      token.text.lower() not in ["meh", "ugh"])]            # Individuell:  Wortfilter
                 for doc in nlp.pipe(df["text"], batch_size=50)]            # Batch-Verarbeitung (50 Texte parallel)

print(f"Verarbeitet: {len(df)} Beschwerden")

# Ausgabe des Prozesses
df.head(10)[["text", "cleaned"]].style.set_properties(
    **{'text-align': 'left', 'width': '1000px', 'max-width': '1500px', 'font-size': '12px'}
)

Verarbeitet: 5627 Beschwerden


,text,cleaned
0,"I used to love Comcast. Until all these constant updates. My internet and cable crash a lot at night, and sometimes during the day, some channels don't even work and on demand sometimes don't play either. I wish they will do something about it. Because just a few mins ago, the internet have crashed for about 20 mins for no reason. I'm tired of it and thinking about switching to Wow or something. Please do not get Xfinity.","['love', 'comcast', 'constant', 'update', 'internet', 'cable', 'crash', 'lot', 'night', 'day', 'channel', 'work', 'demand', 'play', 'wish', 'min', 'ago', 'internet', 'crash', 'min', 'reason', 'tired', 'think', 'switch', 'wow', 'xfinity']"
1,I'm so over Comcast! The worst internet provider. I'm taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service. Definitely switching to Verizon. I'd rather pay $10 extra then dealing w/ Comcast and non stopping internet problems.,"['comcast', 'bad', 'internet', 'provider', 'take', 'online', 'class', 'multiple', 'time', 'late', 'assignment', 'power', 'interruption', 'area', 'lead', 'poor', 'quality', 'internet', 'service', 'definitely', 'switch', 'verizon', 'pay', 'extra', 'deal', 'comcast', 'non', 'stop', 'internet', 'problem']"
2,"If I could give them a negative star or no stars on this review I would. I have never worked with any industry with as bad of customer service as Comcast. It is not a matter of money because I make well enough above and beyond to afford their services but they are a legitimate ripoff. I think they are the biggest scam of since the mortgage industry's major meltdown and I hope I move somewhere where Comcast does not exist. The disregard to want to help or do the right thing is honestly astounding. If you have to call, which you do FOR ALL ISSUES - billing, connection/service, adding or removing service, errors, it does not matter you will be transferred minimum of 4 times. Everyone says the same thing and passes the issues to the next person and no one resolves the problem.They offer promotional packages in small timeframes and can never access them again so they then upgrade you without you wishing and change your billing. It has been 5 months and I have been overcharged $40 a month since I started with them. The blatant rudeness that must make you qualified to do this job is the type of quality service that gets you this review. So... Dear Comcast, you suck. Sincerely, a customer who cannot wait to never use your service again.","['negative', 'star', 'star', 'review', 'work', 'industry', 'bad', 'customer', 'service', 'comcast', 'matter', 'money', 'afford', 'service', 'legitimate', 'ripoff', 'think', 'big', 'scam', 'mortgage', 'industry', 'major', 'meltdown', 'hope', 'comcast', 'exist', 'disregard', 'want', 'help', 'right', 'thing', 'honestly', 'astounding', 'issues', 'billing', 'connection', 'service', 'add', 'remove', 'service', 'error', 'matter', 'transfer', 'minimum', 'time', 'say', 'thing', 'pass', 'issue', 'person', 'resolve', 'problem', 'offer', 'promotional', 'package', 'small', 'timeframe', 'access', 'upgrade', 'wish', 'change', 'billing', 'month', 'overcharge', 'month', 'start', 'blatant', 'rudeness', 'qualified', 'job', 'type', 'quality', 'service', 'get', 'review', 'dear', 'comcast', 'suck', 'sincerely', 'customer', 'wait', 'use', 'service']"
3,"I've had the worst experiences so far since install on 10/4/16. Nothing but problems. Two no shows on scheduled service appointments, extreme difficulty in adding boxes to the second floor. What is so difficult about adding boxes to an existing account? No thank you, I'm not starting a second account for the second floor of the same house! A separate bundle package? All I wanted was just to add a few boxes. Apparently this is not possible. Well then, I guess it's not possible to remain a customer!","['bad', 'experience', 'far', 'install', 'problem', 'show', 'schedule', 'ser

# Vokabularerstellung (engl. vocabulary construction)
Mapping der gefilterten Wörter (Token) zu IDs.

Wortfrequenzschwellen (ist in SpaCy Pipeline)
ggf. nur Nomen?

In [ ]:
# Vokabularerstellung (engl. vocabulary construction)
from sklearn.feature_extraction.text import CountVectorizer

# Umwandung des Dataframes in Liste (Vectorizer braucht Strings)
df["text_cleaned"] = [" ".join(tokens) for tokens in df["cleaned"]]

vectorizer = CountVectorizer()                                                  # Vectorizer mit fit_transform: Vokabular + Matrix in einem
X = vectorizer.fit_transform(df["text_cleaned"])

## Vokabularerstellung
vocabulary = vectorizer.get_feature_names_out()                                 # Vokabular extrahieren
print(f"Vokabulargröße: {len(vocabulary)} Token (Wörter)")                      # Ausgabe der Vokabulargrö0e

word_counts = X.sum(axis=0).A1                                                  # Häufigkeiten (Summe pro Spalte)
vocab_df = pd.DataFrame({
    'word': vocabulary,
    'frequency': word_counts
}).sort_values('frequency', ascending=False)

vocab_df.head(50)                                                               # Erste Spalte ID

Vokabulargröße: 11975 Token (Wörter)


,word,frequency
2176,comcast,15282
9498,service,15181
10576,tell,7876
1735,call,7790
10755,time,6242
2731,customer,5736
1348,bill,5625
5667,internet,5257
9309,say,5219
6905,month,4962


# Textbereinigung (engl. text cleaning)

##  Rauschentfernung (engl. noise reduction)
### Wortbereinigung (engl. word cleaning)
#### Stoppworte (engl. stopwords)
- individuelle Stopworte (Beschwerde)
### Zeichenbereinigung (engl. character cleaning)
#### Satzzeichen (engl. punctuation marks)
#### Leerzeichen (engl. white space)
#### Sonderzeichen (engl. special character)
### Nummernbereinigung (engl. numbers cleaning)
#### Nummern (engl. removing numbers)


##  Standardisierung (engl. standardisation)
### Normalisierung (engl. normalisation)
#### Kasusumwandlung (engl. case conversion)
#### Formatnormalisierungen (engl. format normalisations)
### Rechtschreibfehlerkorrektur (engl. spelling correction)
auf eine Rechtschreikorrektur wird verzichtet, da contextualSpellcheck sicht nicht installieren ließ (Mac M1), T5 zu lange dauerte 10min + und PyEnchant schlechte Korrekturen lieferte

# lexikalische Verarbeitung (engl. lexical processing)
##  Tokenisierung (engl. tokenization)
##  Grundformreduktion (engl. inflection reduction)
### Lemmatisierung (engl. lemmatization)
NLTK(WordNetLemmatizer) / SpaCy
##  Vokabularerstellung/Wortschatzaufbau (engl. vocabulary construction)
Scikit-learn (CountVectorizer)
fit_transform lernt das Vokabular / dict und gibt eine Matrix zurück
# syntaktische Verarbeitung (engl. syntactic processing)
# semantische Verarbeitung (engl. context processing)
## Semantisches Parsen (engl. semantic parsing)
### Eigennamenerkennung (engl. Named Entity Recognition - NER)

# syntaktische Verarbeitung (engl. syntactic processing)
auf ein syntaktische Parsen wrd zunächst verzichtet.

# semantische Verarbeitung (engl. context processing)
ggf. eine NER durchführen

# Vektorisierung (engl. vectorization)
## häufigkeitsbasierter Vektor (engl. frequency vectors)
### TF-IDF

In [ ]:
# Vektorisierung (engl. vectorization)
## häufigkeitsbasierter Vektor (engl. frequency vectors)
### TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Vokabular im korrekten Format: {wort: index}
vocabulary = {word: idx for idx, word in enumerate(vocab_df['word'])}

# Inverse Mapping: {index: wort}
idx_to_word = {idx: word for word, idx in vocabulary.items()}

# TF-IDF mit dem erstellten Vokabular
vectorizer = TfidfVectorizer(vocabulary=vocabulary)

# Texte für TF-IDF vorbereiten (mit den bereinigten Token)
tfidf_matrix = vectorizer.fit_transform(df["text_cleaned"])

print(f"TF-IDF Matrix Form: {tfidf_matrix.shape}\n")

# Sparse-Matrix
result_data = []
cx = tfidf_matrix.tocoo()
for i, j, v in zip(cx.row, cx.col, cx.data):
    result_data.append({
        'Beschwerde'    : i+1,                                                                                      # Beschwerdeindex angepasst
        'ID-Vokabular'  : j,
        'Token (Wort)'  : idx_to_word[j], 
        'TF-IDF Score'  : v
    })

result_df = pd.DataFrame(result_data).sort_values(['Beschwerde', 'TF-IDF Score'], ascending=[True, False])
print(result_df.head(30).to_string(index=False))                                                                # pandas-Spalte ausblenden

TF-IDF Matrix Form: (5627, 11975)

 Beschwerde  ID-Vokabular Token (Wort)  TF-IDF Score
          1          1807        crash      0.566538
          1           352          min      0.379436
          1           793     constant      0.223336
          1           713          wow      0.221154
          1           762         love      0.220104
          1           582        tired      0.206122
          1           451       update      0.197124
          1           403         wish      0.186337
          1           270        night      0.173111
          1           291          lot      0.169078
          1           232       demand      0.168771
          1           187         play      0.159005
          1           200       reason      0.155522
          1             7     internet      0.151246
          1           112      xfinity      0.142833
          1           119       switch      0.134631
          1            55      channel      0.133232
          1

## Worteinbettung (engl. word embeddings)
### Glove (global Vectors for word vectorization)
GloVe steht für „Global Vectors for word vectorization“ (Globale Vektoren für die Wortdarstellung). Es handelt sich hierbei um eine weitere Vektorisierungsmethode, die häufig im NLP verwendet wird, um semantische und syntaktische Informationen in einem Vektorraum darzustellen. Während Word2Vec ein prädiktives Modell ist, ist GloVe ein unüberwachter Ansatz, der auf der Anzahl der Wörter basiert. Sie wurde entwickelt, weil Pennington et al. (2014) zu dem Schluss kamen, dass der Skip-Gram-Ansatz in Word2Vec die statistischen Informationen in Bezug auf das gemeinsame Vorkommen (Kookkurenz) von Wörtern nicht vollständig berücksichtigt. Deshalb haben sie den Skip-Gram-Ansatz mit den Vorteilen der Matrixfaktorisierung kombiniert. Das GloVe-Modell verwendet eine Kookkurenzmatrix, die Informationen über den Wortkontext enthält. Es hat sich gezeigt, dass das entwickelte Modell verwandte Modelle übertrifft, insbesondere bei der Erkennung von benannten Entitäten und Ähnlichkeitsaufgaben (Pennington et al., 2014).

(iu. DLBDSEAIS01_D , 2023, p. 70)

Bias im Datensatz?

In [ ]:
## Worteinbettung mit Word2Vec (engl. word embeddings)
import numpy as np
from gensim.models import Word2Vec

print("Starte Word2Vec-Training...")

# 1. Vorbereitung: Tokenisierte Daten aus df["cleaned"]
sentences = df["cleaned"].tolist()  # Liste von Listen mit Token

# 2. Word2Vec-Modell trainieren
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=200,           # Dimensionalität der Wort-Vektoren
    window=5,                  # Kontextfenster (±5 Wörter)
    min_count=2,               # Wörter die weniger als 2x vorkommen ignorieren
    workers=4,                 # Parallel-Verarbeitung
    sg=0,                      # 0=CBOW, 1=Skip-gram
    epochs=10
)

print(f"✓ Word2Vec-Modell trainiert!")
print(f"  Vokabular-Größe: {len(w2v_model.wv)}")
print(f"  Vektor-Dimension: {w2v_model.vector_size}")

# 3. Embedding-Matrix für das gesamte Vokabular erstellen
embedding_matrix = np.zeros((len(vocab_df), 200))

for idx, word in enumerate(vocab_df['word'].values):
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
    else:
        # Wort nicht im Modell: klein random Vektor
        embedding_matrix[idx] = np.random.randn(200) * 0.01

print(f"✓ Embedding-Matrix erstellt: {embedding_matrix.shape}")

# 4. Dokumentenvektoren erstellen (Durchschnitt aller Wortvektoren)
def get_document_vector(tokens, model, vector_size=200):
    """Berechnet den Durchschnittvektor eines Dokuments"""
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Dokumentenvektoren für alle Beschwerden
document_embeddings = np.array([
    get_document_vector(tokens, w2v_model, 200) 
    for tokens in df["cleaned"]
])

print(f"✓ Dokumentenvektoren erstellt: {document_embeddings.shape}")

# 5. Beispiele: Ähnliche Wörter finden
print("\n--- Ähnliche Wörter (Word2Vec) ---")
test_words = ['internet', 'comcast', 'time']
for word in test_words:
    if word in w2v_model.wv:
        similar = w2v_model.wv.most_similar(word, topn=10)
        print(f"\nÄhnlich zu '{word}':")
        for similar_word, score in similar:
            print(f"  {similar_word}: {score:.3f}")
    else:
        print(f"\n'{word}' nicht im Modell")

Starte Word2Vec-Training...
✓ Word2Vec-Modell trainiert!
  Vokabular-Größe: 7085
  Vektor-Dimension: 200
✓ Embedding-Matrix erstellt: (11975, 200)
✓ Dokumentenvektoren erstellt: (5627, 200)

--- Ähnliche Wörter (Word2Vec) ---

Ähnlich zu 'internet':
  cutter: 0.615
  wifi: 0.613
  dsl: 0.581
  blast: 0.581
  landline: 0.578
  1yr: 0.572
  6mbs: 0.570
  mbps: 0.557
  advertised: 0.538
  100mbps: 0.528

Ähnlich zu 'comcast':
  service: 0.464
  arlington: 0.462
  company: 0.455
  shreveport: 0.453
  inclination: 0.452
  satisfied: 0.451
  lawyers: 0.448
  fraudulently: 0.443
  24h: 0.439
  wth: 0.428

Ähnlich zu 'time':
  hour: 0.682
  countless: 0.633
  dozen: 0.623
  hrs: 0.587
  numerous: 0.577
  phone: 0.575
  half: 0.572
  multiple: 0.562
  occasion: 0.557
  course: 0.550


# Latent Dirichlet Allocation (LDA)

In [25]:
# Latent Dirichlet Allocation (LDA)
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pandas as pd

print("=" * 80)
print("LATENT DIRICHLET ALLOCATION (LDA)")
print("=" * 80)

print("\n1. Überprüfe Datenverfügbarkeit...")

# Prüfe, ob 'cleaned' Spalte existiert
if "cleaned" in df.columns:
    print("   ✓ 'cleaned' Spalte gefunden")
    cleaned_tokens = df["cleaned"].tolist()
else:
    print("   ✗ 'cleaned' Spalte nicht gefunden!")
    print("   Nutze 'text_cleaned' und konvertiere...")
    cleaned_tokens = [tokens.split() for tokens in df["text_cleaned"]]

print(f"   ✓ {len(cleaned_tokens)} Dokumente geladen")

# Überprüfe auf leere Dokumente
non_empty_count = sum(1 for doc in cleaned_tokens if len(doc) > 0)
empty_count = len(cleaned_tokens) - non_empty_count
print(f"   - Nicht-leere Dokumente: {non_empty_count}")
print(f"   - Leere Dokumente: {empty_count}")

# Debug: Zeige erste Tokens
if non_empty_count > 0:
    sample_doc = next((doc for doc in cleaned_tokens if len(doc) > 0), None)
    if sample_doc:
        print(f"   - Beispiel-Tokens (erste 10): {sample_doc[:10]}")

if non_empty_count == 0:
    print("   ✗ FEHLER: Alle Dokumente sind leer!")
else:
    # Entferne leere Dokumente
    cleaned_tokens = [doc for doc in cleaned_tokens if len(doc) > 0]
    print(f"   → Nach Filterung: {len(cleaned_tokens)} nicht-leere Dokumente")
    
    print(f"\n2. Erstelle Dictionary aus Token-Liste...")
    dictionary = Dictionary(cleaned_tokens)
    print(f"   Wörter VOR Filter: {len(dictionary)}")
    
    # Debug: Zeige häufigste Wörter VOR Filter
    print(f"\n   Top 10 häufigste Wörter:")
    word_freq = {}
    for doc in cleaned_tokens:
        for word in doc:
            word_freq[word] = word_freq.get(word, 0) + 1
    
    sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:10]
    for word, freq in sorted_words:
        print(f"     '{word}': {freq}x")
    
    # MINIMALE FILTER: So mild wie möglich
    print(f"\n   Wende MINIMALE Filter an:")
    print(f"   - no_below=1 (Wort kommt mindestens 1x vor)")
    print(f"   - no_above=0.99 (Wort kommt in max. 99% der Dokumente vor)")
    
    initial_dict_size = len(dictionary)
    dictionary.filter_extremes(no_below=1, no_above=0.99, keep_n=100000)
    print(f"   Wörter NACH Filter: {len(dictionary)} (von {initial_dict_size})")
    
    if len(dictionary) == 0:
        print(f"\n   ⚠️ WARNUNG: Dictionary ist leer nach Filter!")
        print(f"   → Verwende Dictionary OHNE Filter...")
        dictionary = Dictionary(cleaned_tokens)
        print(f"   Dictionary (kein Filter): {len(dictionary)} Wörter")
    
    if len(dictionary) > 0:
        print(f"\n3. Erstelle Corpus (Bag-of-Words)...")
        corpus = [dictionary.doc2bow(doc) for doc in cleaned_tokens]
        
        # Debug: Corpus-Statistiken
        corpus_lengths = [len(doc) for doc in corpus]
        non_empty_corpus = sum(1 for doc in corpus if len(doc) > 0)
        
        print(f"   Corpus Größe: {len(corpus)} Dokumente")
        print(f"   Nicht-leere Dokumente im Corpus: {non_empty_corpus}")
        print(f"   Durchschn. Terme pro Dokument: {sum(corpus_lengths)/len(corpus_lengths):.1f}")
        print(f"   Min/Max Terme pro Dokument: {min(corpus_lengths)}/{max(corpus_lengths)}")
        
        # Zeige Sample Corpus-Einträge
        sample_corpus = [doc for doc in corpus if len(doc) > 0][:3]
        print(f"\n   Sample Corpus-Dokumente (erste 3):")
        for i, doc in enumerate(sample_corpus):
            terms = [(dictionary[term_id], freq) for term_id, freq in doc[:5]]
            print(f"     Doc {i}: {len(doc)} Terme - {terms}")
        
        if non_empty_corpus > 0:
            # Entferne Dokumente mit 0 Termen für LDA
            filtered_corpus = [doc for doc in corpus if len(doc) > 0]
            filtered_tokens = [tokens for tokens, doc in zip(cleaned_tokens, corpus) if len(doc) > 0]
            
            print(f"\n4. Trainiere LDA Modell...")
            print(f"   Trainings-Daten: {len(filtered_corpus)} Dokumente mit {len(dictionary)} Wörtern im Dictionary")
            print(f"   Parameter:")
            print(f"   - Topics: 10")
            print(f"   - Passes: 20")
            print(f"   - Iterations: 400")
            print(f"   (Dies kann 5-15 Minuten dauern...)\n")
            
            try:
                model = LdaModel(
                    corpus=filtered_corpus,
                    id2word=dictionary.id2token,
                    num_topics=10,
                    random_state=42,
                    chunksize=2000,
                    passes=20,
                    iterations=400,
                    per_word_topics=True,
                    minimum_probability=0.0,
                    alpha='auto',
                    eta='auto'
                )
                
                print("\n" + "=" * 80)
                print("✓ LDA MODELL ERFOLGREICH TRAINIERT!")
                print("=" * 80)
                print(f"\nModell-Statistiken:")
                print(f"  - Dictionary Größe: {len(dictionary)} Wörter")
                print(f"  - Corpus Größe: {len(filtered_corpus)} Dokumente")
                print(f"  - Anzahl Topics: 10")
                
                # Zeige Top Words pro Topic
                print(f"\n{'='*80}")
                print("TOP WORDS PER TOPIC (mit Gewichtungen)")
                print(f"{'='*80}")
                
                for idx in range(10):
                    terms = model.show_topic(idx, topn=10)
                    print(f"\nTopic {idx}:")
                    for term, weight in terms:
                        print(f"  {term:25s} {weight:.4f}")
                        
            except Exception as e:
                print(f"\n✗ FEHLER beim LDA-Training:")
                print(f"  {type(e).__name__}: {str(e)}")
                import traceback
                traceback.print_exc()
        else:
            print(f"\n✗ FEHLER: Alle Dokumente im Corpus sind leer!")
    else:
        print(f"\n✗ FEHLER: Dictionary ist leer und konnte nicht rekonstruiert werden!")

LATENT DIRICHLET ALLOCATION (LDA)

1. Überprüfe Datenverfügbarkeit...
   ✓ 'cleaned' Spalte gefunden
   ✓ 5627 Dokumente geladen
   - Nicht-leere Dokumente: 5627
   - Leere Dokumente: 0
   - Beispiel-Tokens (erste 10): ['love', 'comcast', 'constant', 'update', 'internet', 'cable', 'crash', 'lot', 'night', 'day']
   → Nach Filterung: 5627 nicht-leere Dokumente

2. Erstelle Dictionary aus Token-Liste...
   Wörter VOR Filter: 12628

   Top 10 häufigste Wörter:
     'comcast': 15215x
     'service': 15169x
     'tell': 7876x
     'call': 7783x
     'time': 6235x
     'customer': 5732x
     'bill': 5619x
     'internet': 5247x
     'say': 5219x
     'month': 4956x

   Wende MINIMALE Filter an:
   - no_below=1 (Wort kommt mindestens 1x vor)
   - no_above=0.99 (Wort kommt in max. 99% der Dokumente vor)
   Wörter NACH Filter: 12628 (von 12628)

3. Erstelle Corpus (Bag-of-Words)...
   Corpus Größe: 5627 Dokumente
   Nicht-leere Dokumente im Corpus: 5627
   Durchschn. Terme pro Dokument: 56.7
  

# BerTopic

In [ ]:
import sklearn
import numpy as np
import pandas as pd
from bertopic import BERTopic

print("=" * 80)
print("BERTOPIC - Topic Modeling")
print("=" * 80)

# Daten vorbereiten

docs = [" ".join(tokens) for tokens in df["cleaned"]]               # Datenaufbereitung für BERTopic (bereinigte Beschwerden)
print(f"   ✓ {len(docs)} Dokumente aus 'cleaned' erstellt")

if docs is not None and len(docs) > 0:
    print(f"\n✓ Daten vorbereitet:")
    print(f"  Anzahl Dokumente (Beschwerden): {len(docs)}")
    print(f"  Erste 10 Beschwerden:")
    for i, doc in enumerate(docs[:10]):
        preview = doc[:80] + "..." if len(doc) > 80 else doc
        print(f"    {i+1}. {preview}")

    print(f"\n2. Starte BERTopic-Training...")
    print("   (Dies kann einige Minuten dauern...)\n")

    topic_model = BERTopic(language="english", calculate_probabilities=True)
    topics, probs = topic_model.fit_transform(docs)

    print(f"\n" + "=" * 80)
    print(f"✓ BERTopic abgeschlossen!")
    print(f"=" * 80)
    print(f"  Topics gefunden: {len(set(topics)) - 1}")  # -1 für Outlier
    print(f"  Dokumente verarbeitet: {len(topics)}")
else:
    print("\n✗ BERTopic konnte nicht trainiert werden (keine Daten)!")


BERTOPIC - Topic Modeling
   ✓ 5627 Dokumente aus 'cleaned' erstellt

✓ Daten vorbereitet:
  Anzahl Dokumente (Beschwerden): 5627
  Erste 10 Beschwerden:
    1. love comcast constant update internet cable crash lot night day channel work dem...
    2. comcast bad internet provider take online class multiple time late assignment po...
    3. negative star star review work industry bad customer service comcast matter mone...
    4. bad experience far install problem show schedule service appointment extreme dif...
    5. check contract sign comcast advertised offer match contract issue sign 150mbps i...
    6. thank god change dish give awesome pricing super people deal actually understand...
    7. long time customer xfinity isp local walmart november customer representative xf...
    8. malfunction dvr manager prevent add recording customer service fairly certain pr...
    9. charge overwhelming comcast service rep ignorant rude resolve issue bill email t...
    10. cable dish verse et

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



✓ BERTopic abgeschlossen!
  Topics gefunden: 40
  Dokumente verarbeitet: 5627


In [57]:
topic_model.get_topic(3)

[('move', 0.04481445961140756),
 ('new', 0.03266076321154964),
 ('address', 0.029590517008694098),
 ('transfer', 0.023903870762157265),
 ('service', 0.022463783873816066),
 ('apartment', 0.021374080065588223),
 ('comcast', 0.020974105574061413),
 ('account', 0.01799278978295429),
 ('call', 0.016336605605041337),
 ('day', 0.01574857058687185)]